In [2]:
import pandas as pd
import numpy as np
import os
import glob
import json
import random
from datetime import datetime
from tqdm import tqdm

In [8]:
def Lorentz(E, *params):
    """
    Compute the Lorentz oscillator dielectric function.

    Parameters:
    - E:     array of photon energies (eV)
    *params: list of parameter values in order [ A1, Br1, E1, A2, Br2, E2...... An, Brn, En]
   
    - An:    oscillator amplitude
    - Br:    broadening parameter
    - En:    resonance energy
    
    Returns:
    - DataFrame with columns:
        • 'Energy (eV)'  
        • 'e1' (real dielectric)  
        • 'e2' (imaginary dielectric)  
        • 'e'  (complex dielectric)
    """

    # 1) Ensure energies are in an array
    E = np.array(E)

    # 1.5 Get parameters from params 

    An = np.array(params[0::3])
    Brn = np.array(params[1::3])
    En = np.array(params[2::3])

    # 2) Compute complex dielectric: ε = ε_inf + (An·Br·En) / (En² − E² − i·Br·E)

    dielectric = np.zeros_like(E) # initalize dielectic function to be 0s

    for Ai, Bri, Ei in zip(An, Brn, En):
    
        numerator   = Ai * Bri * Ei
        denominator = Ei**2 - E**2 - 1j * Bri * E
        dielectric  =  dielectric + numerator / denominator

    # 3) Split into real (e1) and imaginary (e2) parts
    e1 = dielectric.real
    e2 = dielectric.imag

    # 4) Build DataFrame
    df = pd.DataFrame({
        'Photon Energy (eV)': E,
        'e1':          e1,
        'e2':          e2,
        'e':           dielectric
    })

    # 5) Annotate name and source_info for metadata
    """
    Needs to be updated if used in ML
    
    df.name = f"A_{An}_Br_{Br}_En_{En}_Einf_{einf}"
    df.attrs["source_info"] = {
        "model":   "Lorentz",
        "An":      An,
        "Br":      Br,
        "En":      En,
        "Einf":    einf
    }
    """

    return df
# -----------------------------------------------------------------------------

In [10]:
######### Drude Function ######################

def drude(E, *params):
    """
    Compute Drude-model dielectric function for free carriers.

    Parameters:
    - E:       array of photon energies (eV)
    *params: list of parameter values in order [ rho_1, tau_fs1, rho_2, tau_fs2, ......rho_n, tau_fsn]
    
    - rho_n:   material resistivity (Ohm·cm)
    - tau_fs:  carrier scattering time (fs)

    Returns:
    - DataFrame with:
        • 'Energy (eV)'  
        • 'e1' (real part of ε)  
        • 'e2' (imaginary part of ε)  
        • 'e'  (complex dielectric function)
    """

    # 1) Get parameters 

    rhon = np.array(params[0::2])
    taun = np.array(params[1::2])

    
    # 1.1) Physical constants
    hbar     = 6.582119569e-16   # Reduced Planck constant [eV·s]
    eps0     = 8.854e-14         # Vacuum permittivity [F/cm]
    taun      = taun * 1e-15    # Convert scattering time from fs to s

    # 2) Drude numerator & denominator
    #    ε(ω) = -ħ² / [ε₀·ρ_n·(τ·E² + i·ħ·E)]

    dielectric = np.zeros_like(E) # initalize dielectic function to be 0s

    # 3) Compute complex dielectric function
    for rhoi, taui in zip(rhon, taun):
        
        numerator   = -hbar**2
        denominator = eps0 * rhoi * (taui * E**2 + 1j * hbar * E)
        
        dielectric  =  dielectric + numerator / denominator


    # 4) Separate real and imaginary parts
    e1 = dielectric.real
    e2 = dielectric.imag

    # 5) Assemble results into a DataFrame
    df = pd.DataFrame({
        'Photon Energy (eV)': E,
        'e1':          e1,
        'e2':          e2,
        'e':           dielectric
    })

    # 6) Annotate metadata for traceability

    """
    Needs to be updated if used in ML
    
    df.name = f"Drude_rho{rho_n}_tau{tau_fs}"
    df.attrs["source_info"] = {
        "model":   "Drude",
        "rho_n":   rho_n,
        "tau_fs":  tau_fs
    }
    """

    return df


In [15]:
############################### Function that does Sellmeier #######################################

def Sellmeier( A_ir, A_uv, E_uv, e_inf, E):
    """
    Compute the real dielectric function using the Sellmeier model.

    Parameters:
    - A_uv:   UV absorption strE_uvgth coefficiE_uvt
    - A_ir:   IR absorption strE_uvgth coefficiE_uvt
    - E_uv:     resonance E_uvergy (eV)
    - e_inf:  high-frequE_uvcy permittivity
    - E:      array of photon E_uvergies (eV)

    Returns:
    - DataFrame with:
        • 'E_uvergy (eV)'  
        • 'e1' (real dielectric)  
        • 'e2' (imaginary part, zero)  
        • 'e'  (complex dielectric = e1 + 0j)
    """
    # 1) Compute componE_uvts
    UV_dielectric = A_uv / (E_uv**2 - E**2)
    IR_dielectric = -A_ir / (E**2)
    e1 = UV_dielectric + IR_dielectric + e_inf

    # 2) Imaginary part is zero for Sellmeier
    e2 = np.zeros_like(E)
    dielectric = e1 + 0j

    # 3) Build DataFrame
    df = pd.DataFrame({
        'Photon Energy (eV)': E,
        'e1':          e1,
        'e2':          e2,
        'e':           dielectric
    })

    # 4) Add metadata
    df.attrs["source_info"] = {
        "model":  "Sellmeier",
        "A_uv":   A_uv,
        "A_ir":   A_ir,
        "E_uv":     E_uv,
        "e_inf":  e_inf
    }
    df.name = f"Sellmeier_Auv_{A_uv}_Air_{A_ir}_E_uv_{E_uv}_Einf_{e_inf}"

    return df




In [2]:
### Get X axis #####
def get_x_axis(df):
    if 'Wavelength' in df.columns:
        return df['Wavelength (nm)'], 'Wavelength (nm)'
    elif 'Energy (eV)' in df.columns:
        return df['Energy (eV)'], 'Photon Energy (eV)'
    elif 'Photon Energy (eV)' in df.columns:
        return df['Photon Energy (eV)'], 'Photon Energy (eV)'
    else:
        raise ValueError("Neither 'Wavelength' nor 'Photon Energy' found in DataFrame.")

### EMA #####
def Bruggeman_EMA_Roussel(M1, M2, c, name=False):

    
    x_data, x_label = get_x_axis(M1)

    
    N1, N2 = M1['N'].to_numpy(), M2['N'].to_numpy()
    p = N1 / N2
    b = 0.25 * ((3*c - 1) * ((1/p) - p) + p)
    z = b + np.sqrt(b*b + 0.5)
    e = z * N1 * N2
    e1, e2 = e.real, e.imag
    mag = np.sqrt(e1*e1 + e2*e2)
    n = np.sqrt((mag + e1)/2)
    k = np.sqrt((mag - e1)/2)
    df = pd.DataFrame({x_label: x_data, 'e1': e1, 'e2': e2, 'N': n + 1j*k})
    if name:
        df.name = f"EMA_{M1.name}_{M2.name}_{c:.2f}"
    return df

In [4]:

def sumosscilator(df_list):
    """
    Combine multiple oscillator DataFrames into a single composite dielectric.

    Parameters:
    - df_list: list of DataFrames, each with:
        • 'Energy (eV)'
        • 'e1' (real part)
        • 'e2' (imaginary part)

    Returns:
    - DataFrame containing:
        • summed 'e1' and 'e2'
        • complex dielectric 'e = e1 + 1j·e2'
        • preserved 'Energy (eV)' axis
        • metadata listing each component's source_info
    """
    # 1) Use the x axis from the first oscillator

    x_data, x_label = get_x_axis(df_list[0])
    

    # 2) Sum real and imaginary contributions across all oscillators
    e1_total = sum(df["e1"].values for df in df_list)
    e2_total = sum(df["e2"].values for df in df_list)
    e_total  = e1_total + 1j * e2_total

    # 2.5) Calculate n and k 

    n, k = calculate_n_k(e1_total, e2_total)
    N = n + 1j * k

    # 3) Build the composite DataFrame
    df = pd.DataFrame({
        x_label: x_data,
        "e1": e1_total,
        "e2": e2_total,
        "e":  e_total,
        'n': n,
        'k': k,
        'N': N
    })

    # 4) Attach metadata listing each component's model info
    df.attrs["source_info"] = {
        "model":      "Composite",
        "components": [
            d.attrs.get("source_info", {"model": "Unknown"})
            for d in df_list
        ]
    }

    return df

The next set of functions will all be related to the CPPB + Urbach Tail Functions

In [13]:
def CPPB_WVASE(E, *params, derivative=False ):
    """
    Computes ε2(E) using CPPB oscillators. Uses equations provided in CompleteEASE manual.

    Parameters:
    - E: array-like, photon energies in eV
    - *params: flattened list of parameters [A1, Theta1, Br1, E1, mu1, ... An, Thetan, Brn, En, mun]

    A: Amplitude
    Theta: Phase angle in degrees
    Br: Broadening
    E: Resonance Energy 
    Mu: Dimensionality

    Returns:
    a pandas dataframe with:
    - ε2(E): numpy array of the same shape as E
    - ε1(E): numpy array of the same shape as E

    The first and second derivatives of ε1 and ε2 will also be provided if derivative = True
    """

    E = np.array(E)
    # Step 1: Get Parameters 
    
    A     = np.array(params[0::5]) #  start at 0, every 5th parameter is A
    Theta = np.array(params[1::5]) #  start at 1, every 5th parameter is Theta
    Br    = np.array(params[2::5]) #  start at 2, every 5th parameter is Br
    En    = np.array(params[3::5]) #  start at 3, every 5th parameter is En
    mun   = np.array(params[4::5]) #  start at 4, every 5th parameter is mun


    # Step 2: Find bandgap (lowest En)
    E0 = np.min(En)

    # Step 3: Calculate ε2 for E >= E0 (above bandgap)
    e2_total = np.zeros_like(E)
    e1_total = np.zeros_like(E)

    # Fill arrays with 0s for derivatives if needed.
    if derivative:

        e2_d1 = np.zeros_like(E)
        e1_d1 = np.zeros_like(E)
    
        e2_d2 = np.zeros_like(E)
        e1_d2 = np.zeros_like(E)
     

    for Ai, Ti, Bi, Ei, mui in zip(A, Theta, Br, En, mun):

        if mui == 0: 
            
            exp_term =  Ai * np.exp( 1j * Ti * np.pi / 180)
            
            log_term = np.log( 2 * Ei - 2*E - 1j * Bi)
            #print(log_term)
            
            epsilon = exp_term * log_term
            
            e1_total +=  epsilon.real
            
            e2_total +=  epsilon.imag
            
            if derivative:


                numerator_d1 = -1 * exp_term
        
                denominator_d1 = ( Ei - E - 0.5 * 1j * Bi ) 
        
                epsilon_d1 = (numerator_d1 / denominator_d1)
        
                e1_d1 +=  epsilon_d1.real
                e2_d1 +=  epsilon_d1.imag
    
               
                numerator_d2 = -2 * exp_term
        
                denominator_d2 = ( Ei - E - 0.5 * 1j * Bi )**2
        
                epsilon_d1 = (numerator_d1 / denominator_d1)
        
                e1_d2 +=  epsilon_d2.real
                e2_d2 +=  epsilon_d2.imag
           

        else: 
        
            exp_term = np.exp( 1j * Ti * np.pi / 180)
    
            numerator = Ai * exp_term * (0.5 * Bi)** mui
    
            denominator = ( Ei - E - 0.5 * 1j * Bi ) ** mui
    
            epsilon = (numerator / denominator)
    
            e1_total +=  epsilon.real
            e2_total +=  epsilon.imag


            if derivative:
    
                numerator_d1 = mui * Ai * exp_term * (0.5 * Bi)** mui
        
                denominator_d1 = ( Ei - E - 0.5 * 1j * Bi ) ** (mui + 1)
        
                epsilon_d1 = (numerator_d1 / denominator_d1)
        
                e1_d1 +=  epsilon_d1.real
                e2_d1 +=  epsilon_d1.imag
    
                numerator_d2 = mui * (mui + 1) * Ai * exp_term * (0.5 * Bi)** mui
        
                denominator_d2 = ( Ei - E - 0.5 * 1j * Bi ) ** (mui + 2)
        
                epsilon_d2 = (numerator_d2/ denominator_d2)
        
                e1_d2 +=  epsilon_d2.real
                e2_d2 +=  epsilon_d2.imag


    if derivative:

                     
        df = pd.DataFrame({
        'Photon Energy (eV)': E,
        'e1':  e1_total,
        'e2': e2_total,
        'e1_d1': e1_d1,
        'e2_d1': e2_d1,  
        'e1_d2': e1_d2,
        'e2_d2': e2_d2, 
        })

    else:

        df = pd.DataFrame({
        'Energy': E,
        'e1':  e1_total,
        'e2': e2_total,
        })


    return df



##############################################################################################################



def get_CPPB_Urbach( E, *params, derivative=False): 

    """
    Computes ε2(E) using CPPB oscillators and an Urbach tail for E < E0.

    Parameters:
    - E: array-like, photon energies in eV
    - *params: flattened list of parameters [A1, Theta1, Br1, E1, mu1, ... An, Thetan, Brn, En, mun, Eu]

    A: Amplitude
    Theta: Phase angle in degrees
    Br: Broadening
    E: Resonance Energy 
    Mu: Dimensionality
    """


    # Step 1: Get parameters 
    E = np.array(E)

    Eu = params[-1] # get Eu parameter
    param = params[:-1] # remove the Eu value from the others
    
    A     = np.array(param[0::5]) #  start at 0, every 5th parameter is A
    Theta = np.array(param[1::5]) #  start at 1, every 5th parameter is Theta
    Br    = np.array(param[2::5]) #  start at 2, every 5th parameter is Br
    En    = np.array(param[3::5]) #  start at 3, every 5th parameter is En
    mun   = np.array(param[4::5]) #  start at 4, every 5th parameter is mun

    # Step 2: Find bandgap and transition Energy
    Et = En.min() + 0.5 * Eu 
    E0 = Et

    # Step 3: Calculate e2 above transition energy using the "CPPB_WVASE" function.

    cp_mask = E >= E0 # define a mask

    if derivative:

        df = CPPB_WVASE(E, *param, derivative=True) # make sure to input a set of params that does not contain Eu
        e2_total = np.zeros_like(E)
        e2_d1 = np.zeros_like(E)
        e2_d2 = np.zeros_like(E)
        e2_total[cp_mask] = df['e2'][cp_mask]
        e2_d1[cp_mask] = df['e2_d1'][cp_mask]
        e2_d2[cp_mask] = df['e2_d2'][cp_mask]

    else: 

        df = CPPB_WVASE(E, *param, derivative=False)
        e2_total = np.zeros_like(E)
        e2_total[cp_mask] = df['e2'][cp_mask]

    
    # Step 4: Urbach tail for E < E0
    e2_E0 = 0.0
    for Ai, Ti, Bi, Ei, mui in zip(A, Theta, Br, En, mun):
        exp_term = np.exp(1j * Ti * np.pi / 180)
        z_E0 = Ei - E0 - 0.5j * Bi
        e2_E0 +=  ((Ai * exp_term)  * ( 0.5 * Bi /z_E0)** mui).imag

    cp_mask_urbach = E < E0
    e2_total[cp_mask_urbach] += e2_E0 * np.exp((E[cp_mask_urbach] - E0) / Eu)

    # Step 4.5: Calculate derivatives if needed. 
    if derivative: 

        e2_d1[cp_mask_urbach] += e2_E0 * (1/ Eu) * np.exp((E[cp_mask_urbach] - E0) / Eu)
        e2_d2[cp_mask_urbach] += e2_E0 * (1/ Eu)**2 * np.exp((E[cp_mask_urbach] - E0) / Eu)
        
        df = pd.DataFrame({
        'Photon Energy (eV)': E,
        'e2':  e2_total,
        'e2_d1': e2_d1,
        'e2_d2': e2_d2,
            
        })
    
        return (df)

       
    else: 

        df = pd.DataFrame({
        'Energy': E,
        'e2':  e2_total,
            
        })
    
        return (df)



################################## KK integration Function ####################################################


#Try custom build function

def kk_e1_from_e2_ev(energy_ev, e2, E_inf, A_ir, A_uv, Euv ):
    """
    Calculate the real part of the dielectric function ε₁(E) from ε₂(E)
    using the Kramers-Kronig relation over photon energy (in eV).
    
    Parameters:
    -----------
    energy_ev : ndarray
        Photon energies in eV (must be > 0 and sorted).
    e2 : ndarray
        Imaginary part of dielectric function (same length as energy_ev).
    
    Returns:
    --------
    e1 : ndarray
        Real part of dielectric function at each energy point.
    """
    energy_ev = np.array(energy_ev)
    e2 = np.array(e2)
    e1 = np.zeros_like(energy_ev)

    for i, E in enumerate(energy_ev):
        integrand = np.zeros_like(energy_ev)
        for j, Ep in enumerate(energy_ev):
            if i == j:
                integrand[j] = 0.0  # Exclude singularity at E = Ep
            else:
                integrand[j] = Ep * e2[j] / (Ep**2 - E**2)
        
        # Integrate over non-uniform energy axis using trapezoidal rule
        e1[i] = (2 / np.pi) * np.trapz(integrand, energy_ev)

    # Calculate and add sellmeier 
    df = Sellmeier(A_ir, A_uv, Euv, E_inf, energy_ev)
    e1_temp = df['e1'].to_numpy()

    e1 = e1 + e1_temp

    return e1


################# Calculate n and k when needed ##############################

def calculate_n_k(eps1, eps2):
    abs_eps = np.sqrt(eps1**2 + eps2**2)
    n = np.sqrt((abs_eps + eps1) / 2)
    k = np.sqrt((abs_eps - eps1) / 2)
    return n, k



################ CPPB + Urbach with e1 ########################



def get_full_CPPB_Urbach( E, *params):

    """
    Computes ε2(E) using CPPB oscillators and an Urbach tail for E < E0.
    Also computes ε1(E) with KK-integration

    Parameters:
    - E: array-like, photon energies in eV
  - *params: flattened list of parameters [A1, Theta1, Br1, E1, mu1, ... An, Thetan, Brn, En, mun] + Eu, Einf,  A_ir, A_uv, E_uv]

  
    A: Amplitude
    Theta: Phase angle in degrees
    Br: Broadening
    E: Resonance Energy 
    Mu: Dimensionality

    
    Eu: Urbach Energy 
    Einf: Constant additive term used in Sellmeier 
    A_uv: UV pole amplitude 
    E_uv: Uv pole energy
    A_ir: IR pole amplitude


    Returns:
    - ε2(E): numpy array of the same shape as E
    - ε1(E): numpy array of the same shape as E
    """
    
    # Step 1: Determine the average spacing of the energy spectra 

    E_length = len(E) # determine lenght of E
    E_min = E.min() # min value of E
    E_max = E.max() # max value of E

    avg_spacing = (E_max - E_min) / E_length

    # Step 1.1 expand energy range using this average spacing

    E_temp = np.arange(0.001, 7, avg_spacing)


    # Step 1.2 get parameters for get_CPPB_Urbach Function and Sellmeier 

    E_uv = params[-1]
    A_uv  = params[-2]
    A_ir  = params[-3]
    E_inf = params[-4]

    CPPB_params = params[:-4]
   

    # Step 2: Use other CPPB_Urbach model to get e2

    df_e2 = get_CPPB_Urbach(E_temp, *CPPB_params)
    E2 = df_e2['e2'].to_numpy()

    # Step 3: Use KK-integration to get E1
    
    E1 = kk_e1_from_e2_ev( E_temp , E2, E_inf, A_ir, A_uv, E_uv )

    # Step 4: Interpolate E1 

    E1_interp = np.interp(E, E_temp, E1)

    # Step 4.5: Calculate E2 using the correct photon energy range 

    df_e2 = get_CPPB_Urbach(E, *CPPB_params)
    E2 = df_e2['e2'].to_numpy()


    
    #Step 5: reorder to match CompleteEASE 
    eps1 = E1_interp[::-1]
    eps2 = E2[::-1]
    E = E[::-1]

    # Step 5.5 calculate n and k 

    n, k = calculate_n_k(eps1, eps2)

    N = n + 1j * k
    
    # Step 6: export 
    
    df = pd.DataFrame({
    'Photon Energy (eV)': E,
    'e1': eps1,
    'e2': eps2,
    'N': N,
    'n': n, 
    'k': k,
        
    })

    df_inverted = df.iloc[::-1].reset_index(drop=True)


    return ( df_inverted)

 

In [1]:
def get_TL_WVASE( E, *params, derivative = False):

    """
    Computes ε2(E) using Tauc-Loretnz Oscillator

    Parameters:
    - E: array-like, photon energies in eV
    - *params: flattened list of parameters [A1, Eo1, Br1, A2, Eo2, Br2.... An, Eon, Brn, Eg]

    A: Amplitude
    Eo: resonance energy 
    Br: Broadening
    Eg: Tauc Gap Energy
    """

    # Step 1: Get Parameters 

    # Exclude Eg from parameters 
    Eg = params[-1]
    param = params[:-1]

    # Get other params

    A     = np.array(param[0::3], dtype=np.complex128) #  start at 0, every 5th parameter is A
    Eo    = np.array(param[1::3], dtype=np.complex128) #  start at 3, every 5th parameter is En
    Br    = np.array(param[2::3], dtype=np.complex128) #  start at 2, every 5th parameter is Br

    # Calculate E2 above Eg

    E2_mask = E >= Eg # define a mask

    e2_TL = np.zeros_like(E)
    for Ai, Eoi, Bri in zip(A, Eo, Br):

        numerator = (Ai * Eoi * Bri * (E - Eg)**2).real
        denominator = ((E**2 - Eoi**2)**2 + (Bri**2 * E**2)).real
        e2_TL[E2_mask] = e2_TL[E2_mask] + ( ( numerator[E2_mask] / denominator[E2_mask]) * (1 / E[E2_mask]) )


    if derivative: 
    
        e2_TL_d1 = np.zeros_like(E)
        for Ai, Eoi, Bri in zip(A, Eo, Br):

            factors = (Ai * Eoi * Bri * (E - Eg)).real
            numerator_d1 = (Eg * (Eoi**4 - 6 * Eoi**2 * E**2 + 3 * Bri**2 * E**2 + 5* E**4) + Eoi**4 * E + 2 * Eoi**2 * E**3 - E**3 * (Bri**2 + 3 * E**2)).real
            denominator_d1 = (E**2 * ( Eoi**4 - 2 * Eoi**2 * E**2 + Bri**2 * E**2 + E**4 )**2).real

            e2_TL_d1[E2_mask] = e2_TL_d1[E2_mask] +  factors[E2_mask]  * (numerator_d1[E2_mask] / denominator_d1[E2_mask])
            
    
        

    
        e2_TL_d2 = np.zeros_like(E)
        for Ai, Eoi, Bri in zip(A, Eo, Br):

            factors = (Ai * Eoi * Bri).real
            
            term1 =   ( (2 * ( 4*E * (E**2 - Eoi**2) + 2 * Bri**2 * E)**2) / ( (E**2 - Eoi**2)**2 + Bri**2 * E**2)**3 ).real

            term2 =  ( (4 * (E**2 - Eoi**2) + 2 * Bri**2 + 8 * E**2 ) / ( (E**2 - Eoi**2)**2 + Bri**2 * E**2)**2 ).real

            term3 =  (( (E - Eg)**2 * ( term1 - term2) ) / E).real

            term4 = ( ( 2 * ( 2*((E - Eg) / E) - ((E - Eg)**2 / E**2)) * (4*E*(E**2 - Eoi**2) + 2*Bri**2 * E) ) / ( (E**2 - Eoi**2)**2 + Bri**2 * E**2)**2 ).real

            term5 = ((2 * (E-Eg)**2 / E**3) - (4 * (E - Eg) / E**2) + 2/E).real

            term6 = ((E**2 - Eoi**2)**2 + Bri**2 * E**2).real

            term7 = term5 / term6

            e2_TL_d2[E2_mask] = e2_TL_d2[E2_mask] +  factors * ( term3[E2_mask] - term4[E2_mask] + term7[E2_mask])

            
        
   

    if derivative: 

        df = pd.DataFrame({
        'Energy': E,
        'e2':  e2_TL.real,
        'e2_d1': e2_TL_d1.real,
        'e2_d2': e2_TL_d2.real
            
        })

    else: 
         
        df = pd.DataFrame({
        'Energy': E,
        'e2':  e2_TL.real,
            
        })
    

    return df



In [3]:
# Now a CPPB function with an Urbach Tail Background will be made


def get_CPPB_Urbach_TL_Background( E, *params, derivative = False): 

    """
    Computes ε2(E) using CPPB oscillators and an Urbach tail for E < E0.
    This function also has a Tauc-Loretnz background oscillator

    Parameters:
    - E: array-like, photon energies in eV
    - *params: flattened list of parameters [A1, Theta1, Br1, E1, mu1, ... An, Thetan, Brn, En, mun, TL_Amp, TL_Eo, TL_Br, Eu]

    A: Amplitude
    Theta: Phase angle in degrees
    Br: Broadening
    E: Resonance Energy 
    Mu: Dimensionality
    """


    # Step 1: Get parameters 
    E = np.array(E)

    Eu = params[-1] # get Eu parameter
    param = params[:-4] # remove the Eu value and TL values from the others
    # Get TL Values
    TL_Eo = params[-3]
    TL_Br = params[-2]
    TL_Amp = params[-4]
    
    A     = np.array(param[0::5], dtype=np.complex128) #  start at 0, every 5th parameter is A
    Theta = np.array(param[1::5], dtype=np.complex128) #  start at 1, every 5th parameter is Theta
    Br    = np.array(param[2::5], dtype=np.complex128) #  start at 2, every 5th parameter is Br
    En    = np.array(param[3::5], dtype=np.complex128) #  start at 3, every 5th parameter is En
    mun   = np.array(param[4::5], dtype=np.complex128) #  start at 4, every 5th parameter is mun

    # Step 2: Find bandgap and transition Energy
    Et = En.min() + 0.5 * Eu 
    E0 = Et
    TL_Eg = E0 # Tauc Gap

    # Step 3: Calculate e2 above transition energy using the "CPPB_WVASE" function and Tauc-Loretnz

    cp_mask = E >= E0 # define a mask

    TL_params = [TL_Amp, TL_Eo, TL_Br, TL_Eg]

    if derivative:
    
        df1 = CPPB_WVASE(E, *param, derivative=True)
        df2 = get_TL_WVASE( E, *TL_params, derivative=True)

    else: 

        df1 = CPPB_WVASE(E, *param, derivative=False)
        df2 = get_TL_WVASE( E, *TL_params, derivative=False)
        
    
    e2_total = np.zeros_like(E)
    e2_total[cp_mask] = df1['e2'][cp_mask] + df2['e2'][cp_mask]

    if derivative:

        # Calculate derivative above Et 
        e2_d1 = np.zeros_like(E)
        e2_d2 = np.zeros_like(E)

        e2_d1[cp_mask] = df1['e2_d1'][cp_mask] + df2['e2_d1'][cp_mask]
        e2_d2[cp_mask] = df1['e2_d2'][cp_mask] + df2['e2_d2'][cp_mask]
    

    
    # Step 4: Urbach tail for E < E0
    e2_E0 = 0.0
    for Ai, Ti, Bi, Ei, mui in zip(A, Theta, Br, En, mun):
        exp_term = np.exp(1j * Ti * np.pi / 180)
        z_E0 = Ei - E0 - 0.5j * Bi
        e2_E0 +=  ((Ai * exp_term)  * ( 0.5 * Bi /z_E0)** mui).imag
    
    cp_mask_urbach = E < E0
    e2_total[cp_mask_urbach] += e2_E0 * (np.exp((E[cp_mask_urbach] - E0) / Eu)).real


    if derivative: 
    
        #print(Eu)
        e2_d1[cp_mask_urbach] += e2_E0 * ((1/ Eu) * np.exp((E[cp_mask_urbach] - E0) / Eu)).real
        e2_d2[cp_mask_urbach] += e2_E0 * ((1/ Eu)**2 * np.exp((E[cp_mask_urbach] - E0) / Eu)).real

    #Step 5: Return E2 and Energy as a df

        df = pd.DataFrame({
        'Photon Energy (eV)': E,
        'e2':  e2_total,
        'e2_d1': e2_d1,
        'e2_d2': e2_d2,
            
        })

        return (df)

    else: 
            
        df = pd.DataFrame({
        'Energy': E,
        'e2':  e2_total,
            
        })
    
        return (df)
    


In [5]:

def get_full_CPPB_Urbach_TL_Background( E, *params):

    """
    Computes ε2(E) using CPPB oscillators and an Urbach tail for E < E0.
    Also computes ε1(E) with KK-integration

    Parameters:
    - E: array-like, photon energies in eV
  - *params: flattened list of parameters [A1, Theta1, Br1, E1, mu1, ... An, Thetan, Brn, En, mun] + TL_Amp, TL_Eo, TL_Br, Eu Einf,  A_ir, A_uv, E_uv]

  
    A: Amplitude
    Theta: Phase angle in degrees
    Br: Broadening
    E: Resonance Energy 
    Mu: Dimensionality

    
    Eu: Urbach Energy 
    Einf: Constant additive term used in Sellmeier 
    A_uv: UV pole amplitude 
    E_uv: Uv pole energy
    A_ir: IR pole amplitude


    Returns:
    - ε2(E): numpy array of the same shape as E
    - ε1(E): numpy array of the same shape as E
    """
    
    # Step 1: Determine the average spacing of the energy spectra 

    E_length = len(E) # determine lenght of E
    E_min = E.min() # min value of E
    E_max = E.max() # max value of E

    avg_spacing = (E_max - E_min) / E_length

    # Step 1.1 expand energy range using this average spacing

    E_temp = np.arange(0.001, 7, avg_spacing)


    # Step 1.2 get parameters for get_CPPB_Urbach, Tauc_Lorentz, and Sellmeier 

    # Sellmeier
    E_uv = params[-1]
    A_uv  = params[-2]
    A_ir  = params[-3]
    E_inf = params[-4]
    

    CPPB_params = params[:-4]
   

    # Step 2: Use CPPB_Urbach + TL model to get e2

    df_e2 = get_CPPB_Urbach_TL_Background(E_temp, *CPPB_params)
    E2 = df_e2['e2'].to_numpy()

    # Step 3: Use KK-integration to get E1
    
    E1 = kk_e1_from_e2_ev( E_temp , E2, E_inf, A_ir, A_uv, E_uv )

    # Step 4: Interpolate E1 

    E1_interp = np.interp(E, E_temp, E1)

    # Step 4.5: Calculate E2 using the correct photon energy range 

    df_e2 = get_CPPB_Urbach_TL_Background(E, *CPPB_params)
    E2 = df_e2['e2'].to_numpy()


    
    #Step 5: reorder to match CompleteEASE 
    eps1 = E1_interp[::-1]
    eps2 = E2[::-1]
    E = E[::-1]

    # Step 5.5 calculate n and k 

    n, k = calculate_n_k(eps1, eps2)

    N = n + 1j * k
    
    # Step 6: export 
    
    df = pd.DataFrame({
    'Photon Energy (eV)': E,
    'e1': eps1,
    'e2': eps2,
    'N': N,
    'n': n, 
    'k': k,
        
    })

    df_inverted = df.iloc[::-1].reset_index(drop=True)


    return ( df_inverted)


The next set of function are related to generating the SE data

In [ ]:
def Snells_Law(Structure, AOI):
    Nmat = np.stack([df["N"].to_numpy() for df in Structure])
    L, P = Nmat.shape
    angles = np.zeros((L,P), dtype=complex)
    angles[0] = np.radians(AOI)
    for i in range(1, L):
        angles[i] = np.arcsin((Nmat[i-1]/Nmat[i]) * np.sin(angles[i-1]))
    return angles


def fresnel_coefficients(N, angles):
    n1, n2 = N[:-1], N[1:]
    t1, t2 = angles[:-1], angles[1:]
    cos1, cos2 = np.cos(t1), np.cos(t2)
    ds = n1*cos1 + n2*cos2
    dp = n2*cos1 + n1*cos2
    rs = (n1*cos1 - n2*cos2)/ds
    ts = (2*n1*cos1)/ds
    rp = (n2*cos1 - n1*cos2)/dp
    tp = (2*n1*cos1)/dp
    return rs, rp, ts, tp

def Scattering_Matrix(N, angles, d, lam, r, t):
    L, P = N.shape
    d = d[:,None]; lam=lam[None,:]
    E = (2*np.pi/lam)*N[1:-1]*d*np.cos(angles[1:-1])
    prop = np.zeros((L-2,P,2,2), dtype=complex)
    prop[:,:,0,0] = np.exp(-1j*E)
    prop[:,:,1,1] = np.exp( 1j*E)
    interf = np.zeros((L-1,P,2,2), dtype=complex)
    interf[:,:,0,0] = 1/t; interf[:,:,0,1] = r/t
    interf[:,:,1,0] = r/t; interf[:,:,1,1] = 1/t
    S = interf[0]
    for i in range(1, L-1):
        S = S @ prop[i-1] @ interf[i]
    return S


def SE_Sim(Structure, AOI, d, write_data=False, NCS=True):
    wv = Structure[0]['Wavelength (nm)'].to_numpy()
    Nmat = np.stack([df["N"].to_numpy() for df in Structure])
    angles = Snells_Law(Structure, AOI)
    rs, rp, ts, tp = fresnel_coefficients(Nmat, angles)
    Ss = Scattering_Matrix(Nmat, angles, d, wv, rs, ts)
    Sp = Scattering_Matrix(Nmat, angles, d, wv, rp, tp)
    Rp = Sp[:,1,0]/Sp[:,0,0]
    Rs = Ss[:,1,0]/Ss[:,0,0]
    rho = np.conj(Rp/Rs)
    psi = np.arctan(np.abs(rho)).real
    delta = np.unwrap(np.angle(rho))
    Nval = np.cos(2*psi).real
    C = (np.sin(2*psi)*np.cos(delta)).real
    S = (np.sin(2*psi)*np.sin(delta)).real
    if NCS:
        return pd.DataFrame({'Wavelength (nm)': wv, 'N': Nval, 'C': C, 'S': S})
    else:
        return pd.DataFrame({
            'Wavelength (nm)': wv,
            'Psi':   psi*180/np.pi,
            'Delta': delta*180/np.pi
        })



Next function calculates Sigma 

In [7]:
def calculate_sigma_weighted(
    N_model, N_exp,
    C_model, C_exp,
    S_model, S_exp,
    n, m,
    var=0.001
):
    """
    Calculate the weighted sigma fit‐metric:
    
      σ = sqrt(  1/(3n − m)  ∑[ (N_i^m − N_i^e)²/var
                              + (C_i^m − C_i^e)²/var
                              + (S_i^m − S_i^e)²/var ]
            )
    
    where `var` is the denominator (≈0.001) under each squared term.
    
    Parameters
    ----------
    N_model, C_model, S_model : array‐like
        Modeled spectra values.
    N_exp, C_exp, S_exp : array‐like
        Experimental spectra values.
    n : int
        Number of data points (length of each spectrum).
    m : int
        Number of fit parameters.
    var : float, optional
        Denominator under each squared residual (default 0.001).
    
    Returns
    -------
    sigma : float
        The weighted fit‐quality metric.
    """
    # ensure numpy arrays
    N_model = np.asarray(N_model)
    N_exp   = np.asarray(N_exp)
    C_model = np.asarray(C_model)
    C_exp   = np.asarray(C_exp)
    S_model = np.asarray(S_model)
    S_exp   = np.asarray(S_exp)
    
    # element‐wise squared differences, divided by var
    resid_N = ( (N_model - N_exp)  /var   )**2 
    resid_C = ( (C_model - C_exp)  /var   )**2
    resid_S = ( (S_model - S_exp)  /var  )**2
    
    # sum over all points & channels
    total = np.sum(resid_N + resid_C + resid_S)
    
    # divide by degrees of freedom and take sqrt
    sigma = np.sqrt(total / (3*n - m))
    return sigma

 

The next set of functions will focus on improving logistics and visualization for curvfitting 

In [18]:
# Now lets define a function that can group the outputs. 

def group_by_number_sorted(params, fixed_mask, perr):
    """
    Group params into dictionary format with error tracking.
    Input params are assumed to already be sorted by number:
    [A1, T1, Br1, En1, Mu1, A2, T2, ..., MuN, Einf, Eu, Delta]

    Returns:
        dict of {label+number or special: (value, error or None)}
    """
    labels = ["A", "T", "Br", "En", "Mu"]
    num_labels = len(labels)

    total_params = len(params)
    Eu_index = total_params - 5
    einf_index = total_params - 4
    Air_index = total_params - 3
    Auv_index = total_params - 2
    Euv_index = total_params - 1

    num_grouped = total_params - 5
    group_size = num_grouped // num_labels

    # Step 1: Build map of param_index -> error for free parameters
    free_indices = [i for i, is_fixed in enumerate(fixed_mask) if not is_fixed]
    if len(free_indices) != len(perr):
        raise ValueError("Mismatch between number of free params and errors!")

    index_to_error = {idx: err for idx, err in zip(free_indices, perr)}

    # Step 2: Build dictionary directly (since already number-sorted)
    ordered_result = {}

    for i in range(group_size):  # i = number index
        for label_index, label in enumerate(labels):
            param_index = i * num_labels + label_index
            key = f"{label}{i+1}"
            value = params[param_index]
            error = index_to_error.get(param_index, None)
            ordered_result[key] = (value, error)

    # Step 3: Handle Einf, Eu, Delta
    for key, idx in [("Eu", Eu_index), ("Einf", einf_index), ("Air", Air_index), ("Auv", Auv_index), ("Euv", Euv_index)]:
        val = params[idx]
        err = index_to_error.get(idx, None)
        ordered_result[key] = (val, err)

    return ordered_result

In [1]:
# Function to get derivatives when provided e2 from numerical inversion or other tabulated forms.

def e2_derivatives(E, e2):

    """ Inputs e2 spectra, outputs the derivatives """

    x = np.array(E)
    y = np.array(e2)
    
    dy_dx = np.gradient(y,x)
    dy2_dx2 = np.gradient(dy_dx,x)

    df = pd.DataFrame({
    'Energy': x,
    'e2':  y,
    'e2_d1': dy_dx,
    'e2_d2': dy2_dx2,
        
    })

    return df



The next section is functions related to fitting SE data. 

In [2]:
# -------------------------------
# Unpack / Pack functions
# -------------------------------

def unpack_params(flat_params, schema):
    """Convert flat param dict into structured layer objects."""
    structured_layers = []
    
    for layer in schema:
        lname = layer["name"]
        ltype = layer["type"]
        structured = {"name": lname, "type": ltype}
        
        # --- tabulated ---
        if ltype == "tabulated":
            if "thickness" in layer:
                structured["thickness"] = flat_params.get(f"{lname}_Thickness", None)
            structured["data"] = layer["data"]
        
        # --- EMA ---
        elif ltype == "ema":
            structured["thickness"] = flat_params.get(f"{lname}_Thickness", None)
            structured["fraction"] = flat_params.get(f"{lname}_Fraction", None)
            structured["ema"] = layer["ema"]
        
        # --- Oscillator ---
        elif ltype == "oscillator":
            structured["thickness"] = flat_params.get(f"{lname}_Thickness", None)
            structured["oscillators"] = []
            
            for osc in layer["oscillators"]:
                otype = osc["type"]

                # --- CPPB (flexible number of oscillators) ---
                if otype == "CPPB":
                    cppb_params = {}
                    i = 1
                    # detect however many oscillators exist
                    while f"{lname}_CPPB_A{i}" in flat_params:
                        labels = [f"A{i}", f"T{i}", f"Br{i}", f"En{i}", f"Mu{i}"]
                        values = [flat_params[f"{lname}_CPPB_{lab}"] for lab in labels]
                        cppb_params.update(dict(zip(labels, values)))
                        i += 1
                    # tail parameters
                    for lab in [ "TL_Amp", "TL_Eo", "TL_Br", "Eu", "Einf", "Air", "Auv", "En"]:
                        if f"{lname}_CPPB_{lab}" in flat_params:
                            cppb_params[lab] = flat_params[f"{lname}_CPPB_{lab}"]
                    structured["oscillators"].append({"type": "CPPB", "params": cppb_params})

                # --- Lorentz ---
                elif otype == "Lorentz":
                    lorentz_list = []
                    i = 1
                    while f"{lname}_Lorentz_A{i}" in flat_params:
                        labels = [f"A{i}", f"Br{i}", f"E{i}"]
                        values = [flat_params[f"{lname}_Lorentz_{lab}"] for lab in labels]
                        lorentz_list.append(dict(zip(labels, values)))
                        i += 1
                    structured["oscillators"].append({"type": "Lorentz", "params": lorentz_list})

                # --- Sellmeier ---
                elif otype == "Sellmeier":
                    labels = ["Air","Auv","Euv","Einf"]
                    values = [flat_params[f"{lname}_Sellmeier_{lab}"] for lab in labels]
                    structured["oscillators"].append({"type": "Sellmeier", "params": dict(zip(labels, values))})

                # --- Drude ---
                elif otype == "Drude":
                    labels = ["rho1","tau1"]
                    values = [flat_params[f"{lname}_Drude_{lab}"] for lab in labels]
                    structured["oscillators"].append({"type": "Drude", "params": dict(zip(labels, values))})
        
        else:
            raise ValueError(f"Unknown layer type: {ltype}")
        
        structured_layers.append(structured)
    
    return structured_layers


def pack_params(structured_layers, schema):
    """Convert structured layers back into flat param dict."""
    flat = {}
    
    for layer in structured_layers:
        lname = layer["name"]
        ltype = layer["type"]
        
        if ltype == "tabulated":
            if "thickness" in layer:
                flat[f"{lname}_Thickness"] = layer.get("thickness")
        
        elif ltype == "ema":
            flat[f"{lname}_Thickness"] = layer.get("thickness")
            flat[f"{lname}_Fraction"] = layer.get("fraction")
        
        elif ltype == "oscillator":
            flat[f"{lname}_Thickness"] = layer.get("thickness")
            
            for osc in layer["oscillators"]:
                otype = osc["type"]

                # --- CPPB (flexible) ---
                if otype == "CPPB":
                    for k, v in osc["params"].items():
                        flat[f"{lname}_CPPB_{k}"] = v
                
                # --- Lorentz ---
                elif otype == "Lorentz":
                    for lor in osc["params"]:
                        for k, v in lor.items():
                            flat[f"{lname}_Lorentz_{k}"] = v
                
                # --- Sellmeier ---
                elif otype == "Sellmeier":
                    for k, v in osc["params"].items():
                        flat[f"{lname}_Sellmeier_{k}"] = v
                
                # --- Drude ---
                elif otype == "Drude":
                    for k, v in osc["params"].items():
                        flat[f"{lname}_Drude_{k}"] = v
    
    return flat


# --- Dict → Array (for curve_fit) ---
def dict_to_array(param_dict, schema):
    flat_dict = pack_params(unpack_params(param_dict, schema), schema)
    return list(flat_dict.values())


# --- Array → Dict (after curve_fit updates values) ---
def array_to_dict(param_array, schema, template_dict):
    flat_dict = pack_params(unpack_params(template_dict, schema), schema)
    keys = list(flat_dict.keys())
    return dict(zip(keys, param_array))


In [4]:
def SE_Data_Gen(layers, E):
    """
    Input:
        layers - structured layers (from unpack_params)
        E - energy grid (numpy array)

    Output:
        structure - dielectric functions per layer
        thicknesses - list of thicknesses
    """

    num_layers = len(layers)
    thicknesses = []
    structure = [None] * num_layers

    #### ----------------- PASS 1: Oscillator + Tabulated layers ----------------- ####
    for i, layer in enumerate(layers):
        lname, ltype = layer["name"], layer["type"]

        # Thickness (may be None for substrate, etc.)
        thick = layer.get("thickness", None)
        thicknesses.append(thick)

        if ltype == "oscillator":
            eps_contributions = []

            # --- Collect all parameters by oscillator type ---
            drude_params = []
            lorentz_params = []
            sellmeier_params = []
            cppb_params = []

            # flatten parameters for each type
            drude_params = [
                val for osc in layer["oscillators"] if osc["type"] == "Drude"
                for val in osc["params"].values()
            ]

            lorentz_params = [
                val for osc in layer["oscillators"] if osc["type"] == "Lorentz"
                for lor in osc["params"]  # list of dicts
                for val in lor.values()
            ]

            sellmeier_params = [
                val for osc in layer["oscillators"] if osc["type"] == "Sellmeier"
                for val in osc["params"].values()
            ]

            cppb_params = [
                val for osc in layer["oscillators"] if osc["type"] == "CPPB"
                for val in osc["params"].values()
            ]

            # --- Call each oscillator function ONCE per type ---
            if drude_params:
                eps_contributions.append(drude(E, *drude_params))
            if lorentz_params:
                eps_contributions.append(Lorentz(E, *lorentz_params))
            if sellmeier_params:
                eps_contributions.append(Sellmeier(*sellmeier_params, E))
            if cppb_params:
                eps_contributions.append(get_full_CPPB_Urbach_TL_Background(E, *cppb_params))
                

            #print(eps_contributions[0].equals(Bulk))
            #print(lname)
            if len(eps_contributions) == 1: 
                eps_total = eps_contributions[0]
            else:
                eps_total = sumosscilator(eps_contributions)
            
            #print(eps_total.equals(Bulk))
            structure[i] = eps_total

        elif ltype == "tabulated":
            structure[i] = layer["data"]  # TODO: interpolate if needed

        elif ltype == "ema":
            # Placeholder — fill in Pass 2
            structure[i] = None

    #### ----------------- PASS 2: EMA layers ----------------- ####
    for i, layer in enumerate(layers):
        if layer["type"] == "ema":
    
            comp1_idx = layer["ema"]["components"][0]["layer"]
            comp2_idx = layer["ema"]["components"][1]["layer"]
            Mat1 = structure[comp1_idx]
            Mat2 = structure[comp2_idx]
            f2 = layer["fraction"]

            EMA = Bruggeman_EMA_Roussel(Mat1, Mat2, f2)
            structure[i] = EMA

    # Drop Nones from thickness list (substrates, etc.)
    thicknesses = np.array([t for t in thicknesses if t is not None])

    # Step 4: Simulate ellipsometric spectra
    SE_Data = SE_Sim(structure, 64.93, thicknesses, write_data=False, NCS=True)

    # Step 5: return ellipsometric spectra
    return SE_Data


In [6]:
from scipy.optimize import curve_fit
import numpy as np

def fit_SE_with_fixed_params(E, exp_data, schema, param_dict, fixed_mask, bounds=None):
    """
    Fit ellipsometric spectra using SE_Data_Gen with fixed/free parameters.

    Parameters
    ----------
    E : array
        Energy grid.
    exp_data : DataFrame
        Experimental spectra with at least ["Psi", "Delta"] columns.
    schema : dict
        Schema used to unpack/pack parameters.
    param_dict : dict
        Flat dictionary of initial guesses for parameters (packed form).
    fixed_mask : list of bool
        Same length/order as param_dict.values(). True = fixed, False = free.
    bounds : (list, list), optional
        Bounds for all params (full space). Will be reduced to free params.

    Returns
    -------
    result : dict with keys:
        "fitted_params" - free params fitted
        "full_params"   - all params after filling back fixed
        "covariance"    - covariance from curve_fit
    """

    # Flatten initial guesses
    all_keys = list(param_dict.keys())
    all_params = np.array(list(param_dict.values()), dtype=float)

    fixed_mask = np.array(fixed_mask, dtype=bool)
    free_mask = ~fixed_mask

    m = len(free_mask)
    n = len(E)

    free_init = all_params[free_mask]

    def full_params(free_vals):
        """Reconstruct full param vector from free subset."""
        full = np.array(all_params, copy=True)
        full[free_mask] = free_vals
        return full

    def wrapped_model_for_curvefit(E, *free_vals):
        """Called by curve_fit — must return 1D residuals."""
        # rebuild flat dict from free params
        full = full_params(free_vals)
        flat_dict = dict(zip(all_keys, full))

        # convert to structured layers
        layers = unpack_params(flat_dict, schema)

        # generate simulated SE spectra
        sim = SE_Data_Gen(layers, E)  # expected DataFrame

        # compute residuals
                 
        res_N = (sim['N'].to_numpy() - exp_data['N'].to_numpy()) / (3 * n - m)
        res_C = (sim['C'].to_numpy() - exp_data['C'].to_numpy()) / (3 * n - m)
        res_S = (sim['S'].to_numpy() - exp_data['S'].to_numpy()) / (3 * n - m)
        
        residuals = np.concatenate([res_N, res_C, res_S])

        print("Params:", free_vals)
        print("Residual norm:", np.linalg.norm(residuals))
        
        return residuals

    # Prepare bounds for free params only
    if bounds is not None:
        lower_full, upper_full = bounds
        lower_free = np.array(lower_full)[free_mask]
        upper_free = np.array(upper_full)[free_mask]
        bounds_free = (lower_free, upper_free)
    else:
        bounds_free = (-np.inf, np.inf)

    # Dummy ydata: curve_fit compares model output to this
    ydata_dummy = np.zeros_like(
        wrapped_model_for_curvefit(E, *free_init)
    )

    # Call curve_fit
    popt, pcov = curve_fit(
        wrapped_model_for_curvefit,
        E,
        ydata_dummy,
        p0=free_init,
        bounds=bounds_free
    )

    final_params = full_params(popt)
    final_dict = dict(zip(all_keys, final_params))

    return {
        "fitted_params": popt,
        "full_params": final_params,
        "full_dict": final_dict,
        "covariance": pcov
    }


In [8]:
def format_fit_results_with_dict(full_dict, popt, perr, fixed_mask, confidence=0.95):
    """
    Format fitted results using full_dict labels + fit mask.
    
    Parameters
    ----------
    full_dict : dict
        Dictionary of all parameters after fit (from fit_SE_with_fixed_params).
    popt : array
        Optimized values for free params (from curve_fit).
    perr : array
        1σ errors for free params (same order as popt).
    fixed_mask : list of bool
        True = fixed param, False = free param. Must match dict order.
    confidence : float
        Confidence level (e.g., 0.95 for ~2σ errors).
    
    Returns
    -------
    results : dict
        Keys = param labels, values = "value ± error" or "value (fixed)".
    """
    from scipy.stats import norm
    zval = norm.ppf(0.5 + confidence/2)

    all_keys = list(full_dict.keys())
    results = {}

    idx_free = 0
    for i, key in enumerate(all_keys):
        val = full_dict[key]

        if fixed_mask[i]:
            # Fixed parameter
            results[key] = f"{val:.4f} (fixed)"
        else:
            # Free parameter → assign error
            err = perr[idx_free] * zval
            results[key] = f"{val:.4f} ± {err:.4f}"
            idx_free += 1

    # sanity check
    assert idx_free == len(popt), f"Used {idx_free}, expected {len(popt)} free params"

    return results


def pretty_print_fit_results(results):
    """
    Group flat results dict into layers/models and pretty-print.
    
    Parameters
    ----------
    results : dict
        { "param_label": "value ± error" } from format_fit_results_with_dict.
    
    Returns
    -------
    nested : dict
        Nested structure {layer: {model: {param: value_str}}}
    text : str
        Human-readable block of results.
    """
    nested = {}

    for key, val in results.items():
        parts = key.split("_")

        # Case 1: layer only (Thickness, Fraction, etc.)
        if len(parts) == 2:
            layer, param = parts
            nested.setdefault(layer, {})[param] = val

        # Case 2: layer + model + param (e.g., HRT1_Lorentz_A1)
        elif len(parts) == 3:
            layer, model, param = parts
            nested.setdefault(layer, {}).setdefault(model, {})[param] = val

        # Case 3: layer + submodel + subsubmodel + param
        # (for more complex schemas like CST-Bulk_CPPB_A1)
        else:
            layer = parts[0]
            model = "_".join(parts[1:-1])  # everything except last
            param = parts[-1]
            nested.setdefault(layer, {}).setdefault(model, {})[param] = val

    # --- make human-readable text ---
    lines = []
    for layer, models in nested.items():
        lines.append(f"\n=== {layer} ===")
        for key, val in models.items():
            if isinstance(val, dict):
                lines.append(f"  [{key}]")
                for pname, pval in val.items():
                    lines.append(f"    {pname}: {pval}")
            else:
                # direct param under layer
                lines.append(f"  {key}: {val}")

    #text = "\n".join(lines)
    return nested #text


import pandas as pd
from scipy.stats import norm

def format_fit_results_table(full_dict, popt, perr, fixed_mask, confidence=0.95):
    """
    Format fitted results into a DataFrame (easy to print or save as CSV).
    
    Parameters
    ----------
    full_dict : dict
        Dictionary of all parameters after fit (from fit_SE_with_fixed_params).
    popt : array
        Optimized values for free params (from curve_fit).
    perr : array
        1σ errors for free params (same order as popt).
    fixed_mask : list of bool
        True = fixed param, False = free param. Must match dict order.
    confidence : float
        Confidence level (e.g., 0.95 for ~2σ errors).
    
    Returns
    -------
    results_df : pd.DataFrame
        Columns: [Parameter, Value, Error, Status]
    """
    zval = norm.ppf(0.5 + confidence/2)

    all_keys = list(full_dict.keys())
    rows = []

    idx_free = 0
    for i, key in enumerate(all_keys):
        val = full_dict[key]

        if fixed_mask[i]:
            rows.append({
                "Parameter": key,
                "Value": val,
                "Error": None,
                "Status": "Fixed"
            })
        else:
            err = perr[idx_free] * zval
            rows.append({
                "Parameter": key,
                "Value": val,
                "Error": err,
                "Status": "Fitted"
            })
            idx_free += 1

    assert idx_free == len(popt), f"Used {idx_free}, expected {len(popt)} free params"

    results_df = pd.DataFrame(rows)
    return results_df